In [ ]:
import numpy as np
from matplotlib import animation, pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.linalg as linalg
import scipy.stats as stats
import scipy.interpolate as interp
from sklearn.cross_validation import KFold

import warnings
warnings.filterwarnings('ignore')
%pylab inline

In [ ]:
### Read in worm data from SharedData .npz file
Worm = 'GFP'
worm_data = np.load('../SharedData/Worm_Angles/WormAngle'+Worm+'.npz')
print('The loaded npz contains the variables:\n', np.sort([i for i in worm_data]))

G_sig = 'GFP' if Worm == 'GFP' else 'GCaMP'

### Import desired variables
G_Raw = worm_data['G_Raw']
R_Raw = worm_data['R_Raw']
Time = worm_data['Time']
proj_t = worm_data['proj_neural_thetas'].T
print(proj_t.shape)
print(G_Raw.shape)


Thanks for the update — I admit I am surprised that it does so much worse!  

Can I ask how you're setting the ridge parameter (and whether it's being applied to these PCA coefficients as well as the RFP data, or just doing ML for these params)?

Also, will you do one thing and just confirm that MSE is indeed *lower* on the training data?  (That is, this seems to suggest we're overfitting when adding in these regressors, but just to confirm that nothing fishy is going on it would be nice to confirm that we do get — as we'd expect — lower training error with the additional regressors).


In [ ]:
### Determine which time columns have NaNs pick first and last w/o any NaNs
badcols = np.array([x|y for (x,y) in zip(np.isnan(G_Raw).any(axis=0), np.isnan(R_Raw).any(axis=0))])
begin_col = np.arange(0,len(Time))[~badcols][0]
end_col = np.arange(0,len(Time))[~badcols][-1] + 1

### Shave off times from before first and last full column
G_Raw = G_Raw[:,begin_col:end_col]
R_Raw = R_Raw[:,begin_col:end_col]
Time = Time[begin_col:end_col, 0]
proj_t = proj_t[:,begin_col:end_col]


[neuron_length, neuron_time] = np.shape(G_Raw)
print('Neurons:', neuron_length, '\nTime Points:', neuron_time, '\nFrom', Time[0], 's to', Time[-1], 's')

### Fill in NaNs with interpolation
for i in np.arange(len(G_Raw)):
    
    g_bad = np.isnan(G_Raw[i])
    if g_bad.any():
        g_interp = interp.interp1d(Time[~g_bad], G_Raw[i,~g_bad], kind='cubic', assume_sorted=True)
        G_Raw[i][g_bad] = g_interp(Time[g_bad])
    
    r_bad = np.isnan(R_Raw[i])
    if r_bad.any():
        r_interp = interp.interp1d(Time[~r_bad], R_Raw[i,~r_bad], kind='cubic', assume_sorted=True)
        R_Raw[i][r_bad] = r_interp(Time[r_bad])
    
    #print(i)
    
    ### Visualize interpolated points

#     plt.scatter(Time[~r_bad], R_Raw[i,~r_bad], color='blue')
#     plt.plot(Time, R_Raw[i], color='blue', alpha=0.2)
#     plt.scatter(Time[r_bad], R_Raw[i,r_bad], color='red')
#     plt.show()
#     break



# Regressing against all RFP

#### Define ridge regression, maximizing marginal likelihood (see Bishop p.167)

In [ ]:
### Define function to do ridge regression for an individual neuron
def ridge_regress(X, Y, 
                  XTX = None, eigs = None, 
                  converge_required = 0.001, a = 1.0, b = 1.0, printer = False):
    
    ### Initialize
    a_old = -1.0
    N = len(X)

    if XTX is None:
        XTX = np.dot(X.T,X)

    if eigs is None:
        eigs = np.linalg.eigvals(XTX)

    ### Loop until alpha converges
    iterations = 0
    while abs(a_old - a) > converge_required:

        # Sigma = (b*XTX + A)^-1
        Sigma = np.linalg.inv(b*XTX + a*np.eye(len(XTX)))

        # mu = b*Sigma*X.T*Y
        mu = b*np.dot(np.dot(Sigma, X.T), Y)

        gamma = np.sum([(b*i)/(a + b*i) for i in eigs])
        a_new = gamma/np.dot(mu.T,mu)

        error = np.sum((Y - np.dot(X, mu))**2)
        b_new = (N - gamma)/error

        a_old = a
        a = a_new
        b = b_new

        iterations += 1
        if printer:
            print(iterations, "    alpha = ", a, " beta = ", b, " Squared-Error = ", error) 

    conf_int = np.sqrt(np.diag(Sigma)) # See Bishop p.167
    
    ### Return regression weights 'mu', std of weights 'conf_int', squared error of regression 'error'
    return mu, conf_int, error

#### Define (Group) ARD for selecting only relevant weights

In [ ]:
def GroupARD(X, Y, in_group = 1, 
             alpha_max = 100000000, max_iterations = 1000, printer=False):

    N = len(X)
    M = len(X[0])
    
    a = np.repeat(1.0, M) # alphas
    b = 1.0               # Beta = 1/sig^2 

    X1 = X

    deletions = []
    old_alphas = [a]

    for ard_iteration in range(max_iterations):

        # Sigma = (b*XTX + A)^-1
        Sigma = np.linalg.inv(b*np.dot(X1.T,X1) + np.diag(a))

        # mu = b*Sigma*X.T*Y
        mu = b*np.dot(np.dot(Sigma, X1.T), Y)

        gamma = 1.0 - a*np.diag(Sigma)
        group_gamma = np.array([np.sum(gamma[i:i+in_group]) for i in range(0, len(gamma), in_group)])
        mu_squared = mu**2
        group_mu = np.array([np.sum(mu_squared[i:i+in_group]) for i in range(0, len(mu_squared), in_group)])
        a_new = group_gamma/group_mu

        error = np.sum((Y - np.dot(X1, mu))**2)
        b_new = (N - np.sum(gamma))/error

        a = [alpha for alpha in a_new for k in range(in_group)]
        b = b_new

        if printer : print("\nIteration: ", ard_iteration, " beta = ", b, " Squared-Error = ", error)  

        over = [i for i in range(len(a)) if a[i] > alpha_max]
        if over:
            if printer : print("Deletions: ", len(over))
            deletions = [over] + deletions
            X1 = np.delete(X1,over,axis=1)
            a = np.delete(a,over)
        else:
            a_converge = np.sum((a - np.array(old_alphas[-1]))**2)
            if printer : print("Alpha distance = ", a_converge, "   Max alpha = ", np.max(a))
            if a_converge < .00001:
                break
        
        old_alphas.append(a)


    # Recover mu
    for i in deletions:
        for j in i:
            a = np.insert(a,j,-1)
            mu = np.insert(mu,j,0)

    if printer: 
        df = pd.DataFrame(list(zip(a, mu)), columns = ['alpha', 'mu'])
        print("\n", df)
        print("\nDeletions:", np.sum([len(i) for i in deletions])/in_group, 'out of', M/in_group)
        
    return mu, a

# Calculate MSE for different regressions

In [ ]:
def MSE(R_Raw, G_Raw, Time, proj=False, proj_deg = 1, proj_t = [], 
        num_timelag=0, num_timejump=0, kfolds=5, 
        reg_type='all', alpha_max = 100000):

    [neuron_length, neuron_time] = np.shape(G_Raw)
    if proj:
        [num_pc,_] = np.shape(proj_t)
        print('had projections')

    ### Initialize design matrix M ###
    window = num_timelag + num_timejump + 1

    # Matrix of times x neural values in window + pc projections
    if proj:
        M = np.zeros((neuron_time - window + 1, neuron_length*window + proj_deg * num_pc))
        for deg in range(1,proj_deg+1):
            for i in range(neuron_time-window+1):
                M[i,neuron_length*window + (deg-1)*num_pc:neuron_length*window + deg*num_pc] = np.power(proj_t[:,i],deg)
    else: 
        M = np.zeros((neuron_time - window + 1, neuron_length*window))
    
    for i in range(neuron_time - window + 1):
        for j in range(neuron_length):
            j_index = j*window
            M[i][j_index:j_index+window] = R_Raw[j,i:i+window]


    Y_all = G_Raw.T[num_timelag:len(G_Raw.T)-num_timejump]
    Time_all = Time[num_timelag:len(Time)-num_timejump]

    #####################################################

    ### Split data into TRAIN and TEST set ###
    kf = KFold(neuron_time-window+1, n_folds=kfolds, shuffle=False, random_state=None) 

    # Initalize variables
    ARD_mu_R = None; ARD_alpha_R = None; ridge_mu_R = None; standard_mu_R = None
    ridge_MSE_R = None; standard_MSE_R = None; ARD_MSE_R = None; single_MSE_R = None;
    for train_index, test_index in kf:

        ### Define TRAINing data
        TRAIN_Y_all = Y_all[train_index]
        TRAIN_M = M[train_index]
        TRAIN_Time = Time_all[train_index]

        ### Remove mean from Y
        mean_TRAIN_Y = np.mean(TRAIN_Y_all, axis=0)
        TRAIN_Y_all = TRAIN_Y_all - mean_TRAIN_Y

        ### Define TESTing data
        TEST_Y_all = TRAIN_Y_all#Y_all[test_index]
        TEST_M = TRAIN_M#M[test_index]
        TEST_Time = TRAIN_Time#Time_all[test_index]

        ####################################################

        ### Run regressions on TRAIN data ###

        ### ARD
        if reg_type in ['ard', 'all']:

            ### Run GroupARD() on all neruons to get all weights
            if proj:
                ARD_mu = np.zeros((neuron_length, neuron_length*window + proj_deg*num_pc))
                ARD_alpha = np.zeros((neuron_length, neuron_length*window + proj_deg*num_pc))
            else:
                ARD_mu = np.zeros((neuron_length, neuron_length*window))
                ARD_alpha = np.zeros((neuron_length, neuron_length*window))
            for i in range(len(ARD_mu)):
                ARD_mu[i], ARD_alpha[i] = GroupARD(TRAIN_M, TRAIN_Y_all[:,i], in_group=window, alpha_max = alpha_max)
            
            # Save variables
            if ARD_mu_R is None or ARD_alpha_R is None:
                ARD_mu_R = ARD_mu/kfolds
                ARD_alpha_R = ARD_alpha/kfolds
            else:
                ARD_mu_R = ARD_mu_R + ARD_mu/kfolds
                ARD_alpha_R = ARD_alpha_R + ARD_alpha/kfolds
            

        ### Ridge Regression
        if reg_type in ['ridge', 'all','ridge_std']:

            ### Run ridge_regression() on all neruons to get all weights
            if proj:
                ridge_mu = np.zeros((neuron_length, neuron_length*window + proj_deg*num_pc))
            else:
                ridge_mu = np.zeros((neuron_length, neuron_length*window))
            XTX = TRAIN_M.T @ TRAIN_M
            eigs = np.linalg.eigvals(XTX)

            for i in range(len(ridge_mu)):
                ridge_mu[i] = ridge_regress(TRAIN_M, TRAIN_Y_all[:,i], XTX = XTX, eigs = eigs)[0]
            if ridge_mu_R is None:
                ridge_mu_R = ridge_mu/kfolds
            else:
                ridge_mu_R = ridge_mu_R + ridge_mu/kfolds

        ### Standard Regression
        if reg_type in ['standard', 'all','ridge_std']:
            try: 
                standard_mu = np.linalg.solve(TRAIN_M.T @ TRAIN_M , TRAIN_M.T @ TRAIN_Y_all).T
            except np.linalg.LinAlgError:
                print("Singular matrix in standard regression, trying least squares solve")
                standard_mu = np.linalg.lstsq(TRAIN_M.T @ TRAIN_M , TRAIN_M.T @ TRAIN_Y_all)[0].T
                
            if standard_mu_R is None:
                standard_mu_R = standard_mu/kfolds
            else:
                standard_mu_R = standard_mu_R + standard_mu/kfolds

        ####################################################

        ### Compute MSE on TEST data ###

        ### Standard regression on single neuron
        #single_MSE = np.zeros((neuron_length)); standard_MSE = None; ridge_MSE = None
        #for i in range(neuron_length):
        #    slope, intercept = stats.linregress(R_Raw[i,:time_index_split],G_Raw[i,:time_index_split])[:2]
        #    single_MSE[i] = np.average((slope*R_Raw[i,time_index_split:] + intercept - G_Raw[i,time_index_split:])**2)
        
        #if single_MSE_R is None:
        #    single_MSE_R = single_MSE/kfolds
        #else:
        #    single_MSE_R = single_MSE_R + single_MSE/kfolds

        ### Ridge regression on all neurons
        if reg_type in ['ridge', 'all', 'ridge_std']:
            ridge_MSE = np.zeros((neuron_length))
            for i in range(neuron_length):
                ridge_MSE[i] = np.average(((TEST_M @ ridge_mu[i]) + mean_TRAIN_Y[i] - TEST_Y_all.T[i])**2)
            
            if ridge_MSE_R is None:
                ridge_MSE_R = ridge_MSE/kfolds
            else:
                ridge_MSE_R = ridge_MSE_R + ridge_MSE/kfolds
                
        ### Standard regression on all neurons
        if reg_type in ['standard', 'all', 'ridge_std']:
            standard_MSE = np.zeros((neuron_length))
            for i in range(neuron_length):
                standard_MSE[i] = np.average(((TEST_M @ standard_mu[i]) + mean_TRAIN_Y[i] - TEST_Y_all.T[i])**2)

            if standard_MSE_R is None:
                standard_MSE_R = standard_MSE/kfolds
            else:
                standard_MSE_R = standard_MSE_R + standard_MSE/kfolds
                
        ### Group ARD on all neurons
        if reg_type in ['ard', 'all']:
            ARD_MSE = np.zeros((neuron_length))
            for i in range(neuron_length):
                ARD_MSE[i] = np.average(((TEST_M @ ARD_mu[i]) + mean_TRAIN_Y[i] - TEST_Y_all.T[i])**2)
                
            if ARD_MSE_R is None:
                ARD_MSE_R = ARD_MSE/kfolds
            else:
                ARD_MSE_R = ARD_MSE_R + ARD_MSE/kfolds

        ####################################################

    return {'ridge_mu' : ridge_mu_R, 'standard_mu' : standard_mu_R, 'ARD_mu' : ARD_mu_R,
            'single_MSE' : single_MSE_R, 'ridge_MSE' : ridge_MSE_R, 'standard_MSE' : standard_MSE_R, 'ARD_MSE' : ARD_MSE_R,
            #'M' : M, 'Y_all' : Y_all, 'Time_all' : Time_all, 
            #'time_index_split' : time_index_split, 'mean_TRAIN_Y' : mean_TRAIN_Y,
            'ARD_alpha' : ARD_alpha_R}

In [ ]:
### Unpack output from MSE()
outProj1 = MSE(R_Raw,G_Raw,Time,proj = True, proj_t = proj_t, proj_deg = 1, reg_type='ridge_std',num_timelag=0, alpha_max=10000)
print("ridge_MSE %3.3f, standard_MSE %3.3f" %(np.mean(outProj1['ridge_MSE']), np.mean(outProj1['standard_MSE'])))

In [ ]:
outProj2 = MSE(R_Raw,G_Raw,Time,proj = True, proj_t = proj_t, proj_deg = 2, reg_type='ridge_std',num_timelag=0, alpha_max=10000)
print("ridge_MSE %3.3f, standard_MSE %3.3f" %(np.mean(outProj2['ridge_MSE']), np.mean(outProj2['standard_MSE'])))

In [ ]:
out = MSE(R_Raw,G_Raw,Time,proj = False, proj_t = proj_t, proj_deg = 2, reg_type='ridge_std',num_timelag=0, alpha_max=10000)
print("ridge_MSE %3.3f, standard_MSE %3.3f" %(np.mean(out['ridge_MSE']), np.mean(out['standard_MSE'])))

In [ ]:
out = MSE(R_Raw,G_Raw,Time,proj = False, proj_t = proj_t, proj_deg = 2, reg_type='ridge_std',num_timelag=0, alpha_max=10000)
print("ridge_MSE %3.3f, standard_MSE %3.3f" %(np.mean(out['ridge_MSE']), np.mean(out['standard_MSE'])))

In [ ]:
f = plt.figure(figsize=(20,10))
plt.plot(out['ridge_MSE'])
plt.plot(outProj1['ridge_MSE'])
plt.plot(outProj2['ridge_MSE'])
plt.xlabel('Neurons', size=14)
plt.ylabel('MSE',size=14)
plt.title('Ridge Regression',size=20)
plt.legend(['Neural activity','Neural activity and projections (deg 1)', 'Neural activity and projections (deg 2)'])

In [ ]:
f = plt.figure(figsize=(20,10))
plt.plot(out['standard_MSE'])
plt.plot(outProj1['standard_MSE'])
plt.plot(outProj2['standard_MSE'])
plt.xlabel('Neurons', size=14)
plt.ylabel('MSE',size=14)
plt.title('Standard Regression',size=20)
plt.legend(['Neural activity','Neural activity and projections (deg 1)', 'Neural activity and projections (deg 2)'])

In [ ]:
print(ridge_MSE/standard_MSE)

In [ ]:
# Looking at how many timesteps back to take. 
results = []
errors = []
for i in range(3):
    temp = MSE(R_Raw,G_Raw,Time, proj_t, alpha_max=100000, num_timelag=i, num_timejump=i)
    results.append(temp)
    errors.append(temp['ridge_MSE']/temp['single_MSE'])
    
print(pd.DataFrame(np.array(errors).T))

In [ ]:
plt.hist(errors[0], bins=60, range = (0.5,3.5), color = 'red', alpha=0.5, cumulative=True)
plt.hist(errors[1], bins=60, range = (0.5,3.5), color = 'green', alpha=0.5, cumulative=True)
plt.hist(errors[4], bins=60, range = (0.5,3.5), color = 'blue', alpha=0.5, cumulative=True)
plt.show()

In [ ]:
for i in range(len(errors)):
    print(i, np.mean(errors[i]), np.median(errors[i]))

In [ ]:
plt.plot(standard_mu[0], color = 'blue')
plt.plot(ridge_mu[0], color = 'green')
plt.plot(ARD_mu[0], color = 'red')
plt.show()

## Extra Functions!

* Look at distribution of NaNs in original data

In [ ]:
### Get list of columns with any NaN present
badcols = np.array([x|y for (x,y) in zip(np.isnan(G_Raw).any(axis=0), np.isnan(R_Raw).any(axis=0))])

### NaN Distribution Information
pd.set_option('display.max_rows', None)

print(len(np.nonzero(badcols)[0]), 'columns/times containing NaNs were found')

print('\nNaN Count by Time Column, Out of', len(G_Raw), 'Neurons')
print(pd.DataFrame(np.column_stack((Time[badcols],np.sum(np.isnan(G_Raw[:,badcols]), axis=0),np.sum(np.isnan(R_Raw[:,badcols]), axis=0))), 
                   index = np.nonzero(badcols)[0], columns=['Time', 'G_Raw', 'R_Raw']))

print('\nNaN Count by Neruon, Out of', len(G_Raw[0]), 'Time Points')
print(pd.DataFrame(np.column_stack((np.sum(np.isnan(G_Raw), axis=1),np.sum(np.isnan(R_Raw), axis=1))), 
                   columns=['G_Raw', 'R_Raw']))

* Remove all columns containing any NaN, instead of interpolating

In [ ]:
### Remove Columns containing NaNs
badcols = np.array([x|y for (x,y) in zip(np.isnan(G_Raw).any(axis=0), np.isnan(R_Raw).any(axis=0))])

G_Raw = G_Raw[:,~badcols]
R_Raw = R_Raw[:,~badcols]
Time = Time[~badcols]

[neuron_length, neuron_time] = np.shape(G_Raw)

* Make a heat map of GFP/GCaMP & RFP

In [ ]:
### Heat Map of GFP/GCaMP & RFP across trial
f ,ax = plt.subplots(1,2, figsize=(16,10*2))
ax[0].imshow(G_Raw, aspect=10,cmap="Paired")
ax[1].imshow(R_Raw, aspect=10,cmap="Paired")
for axis, title in zip(ax, [G_sig, r'RFP']):
    axis.set_title(title)
    axis.set_xlabel(r'Frame #')
    axis.set_ylabel(r'Neuron #')

* Examine Raw Fluorescence statistics within worm

In [ ]:
### Visualize distribution of Raw fluorescence statistics across neurons
p = 8 # fluorescence percentile

f, ax = plt.subplots(2,4, figsize=(20,10))

ax[0][0].hist(np.min(G_Raw, axis=1), color='green')
ax[0][1].hist(np.percentile(G_Raw, p, axis=1), color='green')
ax[0][2].hist(np.median(G_Raw, axis=1), color='green')
ax[0][3].hist(np.max(G_Raw, axis=1), color='green')

ax[1][0].hist(np.min(R_Raw, axis=1), color='red')
ax[1][1].hist(np.percentile(R_Raw, p, axis=1), color='red')
ax[1][2].hist(np.median(R_Raw, axis=1), color='red')
ax[1][3].hist(np.max(R_Raw, axis=1), color='red')

ax[0][0].set_title(G_sig + ', Minimum Raw', fontsize = 16)
ax[0][1].set_title(G_sig + ', ' + str(p) + 'th Percentile Raw', fontsize = 16)
ax[0][2].set_title(G_sig + ', Median Raw', fontsize = 16)
ax[0][3].set_title(G_sig + ', Maximum Raw', fontsize = 16)
ax[1][0].set_title('RFP, Minimum Raw', fontsize = 16)
ax[1][1].set_title('RFP, ' + str(p) + 'th Percentile Raw', fontsize = 16)
ax[1][2].set_title('RFP, Median Raw', fontsize = 16)
ax[1][3].set_title('RFP, Maximum Raw', fontsize = 16)

plt.tight_layout()
plt.savefig('../Data/Worm'+Worm+'Raw_Fluorescence_Dist.pdf')

* Visualize raw activity (optionally rescaled) of particular neruons

In [ ]:
### Visualize Raw Activity of specific neruon (optionally rescaled to 0-1)
def rescale(a):
    return [(i-np.min(a))/(np.max(a)-np.min(a)) for i in a]

n = 12
f = plt.figure(figsize=(20,10))

plt.plot(Time, rescale(G_Raw[n]), color = 'green')
plt.plot(Time, rescale(R_Raw[n]), color = 'red')

# plt.plot(Time, G_Raw[n], color = 'green')
# plt.plot(Time, R_Raw[n], color = 'red')

plt.xlim([0,Time[-1]])

plt.show()
print('Neuron: ', n)
print('            Min       %  Median     Max')
print('G_Raw: ', '%7.1f' % np.min(G_Raw[n]), '%7.1f' % np.percentile(G_Raw[n],8), '%7.1f' % np.median(G_Raw[n]), '%7.1f' % np.max(G_Raw[n]))
print('R_Raw: ', '%7.1f' % np.min(R_Raw[n]), '%7.1f' % np.percentile(R_Raw[n],8), '%7.1f' % np.median(R_Raw[n]), '%7.1f' % np.max(R_Raw[n]))

* Do basic linear regression of GFP/GCaMP against RFP

In [ ]:
### Caclualte regression info of GFP/GCaMP against RFP
slopes = np.zeros(neuron_length)
r_values = np.zeros(neuron_length)
intercepts = np.zeros(neuron_length)

for i in range(neuron_length):
    slope, intercept, r_value, p_value, std_err = stats.linregress(R_Raw[i],G_Raw[i])
    slopes[i] = slope
    r_values[i] = r_value
    intercepts[i] = intercept

* Create Coefficeint of Varaiation figure for RFP vs GFP w/ r^2 as colorbar

In [ ]:
### Display Coefficeint of Variation for RFP vs GFP w/ r^2 as colorbar
def onpick3(event):
    ind = event.ind
    print('onpick3 scatter:', ind)

f = plt.figure(figsize=(10,10))
plt.scatter(stats.variation(R_Raw, axis=1),stats.variation(G_Raw, axis=1), c = r_values, cmap='hot', picker=True)
plt.scatter(np.var(R_Raw, axis=1),np.var(G_Raw, axis=1), c = r_values, cmap='hot', picker=True)
plt.plot(np.arange(0.01,0.6,.01),np.arange(0.01,0.6,.01))
plt.xlim([0,.6])
plt.ylim([0,.6])
plt.xlabel('Cof. of Var. RFP Raw')
plt.ylabel('Cof. of Var. GFP Raw')
plt.colorbar(label=r'$r^2$')

slope, intercept, r_value, p_value, std_err = stats.linregress(stats.variation(R_Raw, axis=1),stats.variation(G_Raw, axis=1))
print(slope, intercept, r_value**2, p_value, std_err)
plt.plot(np.arange(0.01,0.6,.01), intercept + slope*np.arange(0.01,0.6,.01), 'g--')

f.canvas.mpl_connect('pick_event', onpick3)
plt.show()

#plt.savefig('Coeffcient_of_Variation.pdf')

* Visualize fit of weights to neural activity

In [ ]:
### Define which weights to use & neruons to see
weights = ridge_mu
neuron_n = [52,55,12,42]  # List of neurons to be observed

f ,ax = plt.subplots(len(neuron_n),2, figsize=(16,4*len(neuron_n)))
for i,j in enumerate(neuron_n):
#     ax[i][0].plot(Time_all, Y_all.T[j]) # Actual activity
#     ax[i][0].plot(Time_all, (M @ weights[j])) # Recovered Activity using Estimated Weights
#     ax[i][1].plot(Time_all, (M @ weights[j]) - Y_all.T[j]) # Error
    
    ax[i][0].plot(Time_all[time_index_split:], Y_all.T[j,time_index_split:]) # Actual activity
    ax[i][0].plot(Time_all[time_index_split:], (M @ weights[j] + mean_TRAIN_Y[j])[time_index_split:]) # Recovered Activity using Estimated Weights
    #ax[i][0].plot(Time_all[time_index_split:], (M @ ARD_mu[j] + mean_TRAIN_Y[j])[time_index_split:]) # Recovered Activity using Estimated Weights
    ax[i][1].plot(Time_all[time_index_split:], ((M @ weights[j]) - Y_all.T[j] + mean_TRAIN_Y[j])[time_index_split:]) # Error
    
    ax[i][0].set_ylabel(r'Neuron ' + str(neuron_n[i]))
    
for axis, title in zip(ax[0], [r'Actual Activity & Estimated Activity', r'Error']):
    axis.set_title(title)
    
plt.tight_layout()
plt.show()

In [ ]:
### Define which weights to use & neruons to see
weights = ridge_mu
neuron_n = [12,42]  # List of neurons to be observed

f ,ax = plt.subplots(len(neuron_n),1, figsize=(12,4*len(neuron_n)))
for i,j in enumerate(neuron_n):
#     ax[i][0].plot(Time_all, Y_all.T[j]) # Actual activity
#     ax[i][0].plot(Time_all, (M @ weights[j])) # Recovered Activity using Estimated Weights
#     ax[i][1].plot(Time_all, (M @ weights[j]) - Y_all.T[j]) # Error
    
    ax[i].plot(Time_all[time_index_split:], Y_all.T[j,time_index_split:], label='Actual Activity') # Actual activity
    ax[i].plot(Time_all[time_index_split:], (M @ weights[j] + mean_TRAIN_Y[j])[time_index_split:], label='Predicted Activity') # Recovered Activity using Estimated Weights
 
    ax[i].set_title(r'Neuron #' + str(neuron_n[i]))
    ax[i].set_xlabel(r'Time (s)')
    ax[i].set_ylabel(r'Raw GFP Fluorescence')
    ax[i].legend()
    
plt.tight_layout()
plt.show()

In [ ]:
a = np.arange(5)

In [ ]:
a[0:4]